# Recommender system for books: Modelisation

In [1]:
# Data manipulation
import pandas as pd 
import numpy as np 
import gc

# Graphical libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Modelisation libraries
from surprise import BaselineOnly, SVD, NMF, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering, SlopeOne
from surprise import accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import Dataset
from surprise import Reader

from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer

# GHG Emissions tracking
from codecarbon import EmissionsTracker

In [2]:
path = 'Datasets/'

In [3]:
# Importing data about users and books 
users = pd.read_parquet(path+"users_data_cleaning.parquet.gzip")
books = pd.read_parquet(path+"books_data_cleaning.parquet.gzip")

In [4]:
books.rename(columns={"Rating" : "Average_Rating"},
             inplace=True)

It exists different kinds of recommender engines. In this proof of concept, simple recommender, user based collaborative filtering, content-based recommender and hybrid methods will be testing to find the best recommender engines. 

## 1) Popularity based

Simple recommender is the most basic recommender engines. It based on popularity. To calcule it, rating and number of reviews are used. The formula used is detailled in Bhowmick et al. (2021).

In [5]:
# creating a new dataset for the calculus
simple_reco = books.copy()
simple_reco = simple_reco[["Id", "Name", "Authors",
                           "Average_Rating", "CountsOfReview"]]
simple_reco.head()

,Id,Name,Authors,Average_Rating,CountsOfReview
0,4000063,The Flintstones in Viva Rock Vegas,Ellen Miles,3.82,1
1,4000100,Little Rhody,Neta Lohnes Frazier,4.33,1
2,4000228,Finance And Investments Using The Wall Street ...,Peter R. Crabb,0.00,0
3,4000366,Shorty McCabe Looks 'Em Over,Sewell Ford,0.00,0
4,4000441,Plant Pathology,George N. Agrios,4.52,0


In [6]:
# calculating the average rating
C = simple_reco['Average_Rating'].mean()
print(C)

3.7320950652695215


In [7]:
# calculating and keeping
# books with the 90th percentile
m = simple_reco['CountsOfReview'].quantile(0.90)
print(m)

41.0


In [8]:
# selecting only books with at least 41 reviews
q_books = simple_reco.copy().loc[simple_reco['CountsOfReview'] >= m]
q_books.shape

(12116, 5)

In [9]:
def weighted_rating(x, m=m, C=C):
    v = x['CountsOfReview']
    R = x['Average_Rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [10]:
q_books['score'] = q_books.apply(weighted_rating, axis=1)

In [11]:
#Sort movies based on score calculated above
q_books = q_books.sort_values('score',
                              ascending=False)

#Print the top 5 books
q_books[["Id",
         "Name",
         "Authors",
         "CountsOfReview", 
         "Average_Rating", 
         "score"]].head(5)

,Id,Name,Authors,CountsOfReview,Average_Rating,score
44289,862041,"Harry Potter Series Box Set (Harry Potter, #1-7)",J.K. Rowling,6522,4.74,4.733703
40413,818056,Harry Potter and the Deathly Hallows (Harry Po...,J.K. Rowling,952,4.62,4.583339
115327,1215032,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,16523,4.57,4.567926
61446,3165162,Percy Jackson and the Olympians (Percy Jackson...,Rick Riordan,546,4.59,4.530078
72565,1025685,"The Absolute Sandman, Volume Two",Neil Gaiman,198,4.69,4.525673


In [12]:
print(round(np.sqrt(mean_squared_error(q_books['Average_Rating'],
                                 q_books["score"])), 3))

0.108


In [13]:
del q_books, simple_reco
gc.collect()

0

This method had disadvantage to make recommendation based on the most popular books. Books with few ratings have less change to be recommend. 

## 2) User-based collaborative filtering

### a) Preparing the data for Surprise library

In [14]:
data = pd.merge(users, books, on = "Id")
data.rename(columns={"Name" : "Title",
                     "Id" : "Book_Id"}, 
                     inplace=True)

In [15]:
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(data[['User_Id', 'Book_Id', 'Rating']], reader)

In [16]:
trainset, testset = train_test_split(data_surprise, test_size=.2)

In [17]:
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
base_als = BaselineOnly(bsl_options=bsl_options)
predictions = base_als.fit(trainset).test(testset)

Estimating biases using als...


In [18]:
del predictions
gc.collect()

0

In [19]:
bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }
base_sgd = BaselineOnly(bsl_options=bsl_options)
predictions = base_sgd.fit(trainset).test(testset)

Estimating biases using sgd...


In [20]:
del predictions
gc.collect()

0

### b) Collaborative filtering Algorithms

In [21]:
# Initializing similarities options
sim_options = {'name': 'cosine',
               'user_based': True
               }

In [22]:
svd = SVD()
nmf = NMF()
knn = KNNBasic(random_state=42)
slope = SlopeOne()
cocluster = CoClustering(random_state=42)

In [23]:
model = [svd, nmf, knn, slope, cocluster]
dict_model = {}

for model in model:
    tracker = EmissionsTracker()
    tracker.start()
    y_pred = model.fit(trainset).test(testset)
    emissions = tracker.stop()
    rmse = accuracy.rmse(y_pred)
    dict_model[model] = (rmse.round(3), emissions)

eval_model = pd.DataFrame(dict_model, index=["RMSE", "GHG Emissions"])
eval_model.columns = ["SVD", "NMF", "KNN", "SlopeOne", "Co-Clustering"]

eval_model

/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


RMSE: 0.7652
RMSE: 0.8745
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7700


/tmp/ipykernel_38731/4219558851.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred = model.fit(trainset).test(testset)


RMSE: 0.8908
RMSE: 0.8908


,SVD,NMF,KNN,SlopeOne,Co-Clustering
RMSE,0.765000,0.875000,0.770000,0.891000,0.891000
GHG Emissions,0.000027,0.000029,0.000015,0.000034,0.000015


In [24]:
del dict_model,\
    sim_options
gc.collect()

0

### c) Focus on K-NN based models

In [25]:
# Initializing similarities options
sim_options = {'name': 'cosine',
               'user_based': True
               }

In [26]:
knnbasic = KNNBasic()
knnmeans = KNNWithMeans()
knnzscore = KNNWithZScore()

In [27]:
model = [knnbasic, knnmeans, knnzscore]
dict_model = {}

for model in model:
  model = [knnbasic, knnmeans, knnzscore]
dict_model = {}

for model in model:
    tracker = EmissionsTracker()
    tracker.start()
    y_pred = model.fit(trainset).test(testset)
    emissions = tracker.stop()
    rmse = accuracy.rmse(y_pred)
    dict_model[model] = (rmse.round(3), emissions)

eval_model = pd.DataFrame(dict_model, index=["RMSE", "GHG Emissions"])
eval_model.columns = ["KNN Basic", "KNN Means", "KNN Z-Score"]

eval_model

/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7700
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7694
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7620


,KNN Basic,KNN Means,KNN Z-Score
RMSE,0.770000,0.769000,0.762000
GHG Emissions,0.000022,0.000013,0.000014


In [28]:
param_grid = {'k': [20, 30, 40, 50, 60, 70, 80]}

gs = GridSearchCV(KNNWithZScore,
                  param_grid,
                  measures=['rmse', 'mae'],
                  cv=5)

gs.fit(data_surprise)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [29]:
knn_gs = gs.best_estimator['rmse']

tracker = EmissionsTracker()

tracker.start()
predictions = knn_gs.fit(trainset).test(testset)
emissions = tracker.stop()


accuracy.rmse(predictions)
print("GHG emission: " + str(emissions) + " kg/CO2")

/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/home/cecile/anaconda3/envs/nlpenv/lib/python3.9/site-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7443
GHG emission: 2.7803216914243218e-05 kg/CO2


In [30]:
del gs,\
    dict_model,\
    predictions,\
    tracker,\
    emissions
gc.collect()

88

## 3) Content-based algorithms

### a) With Tfidf

https://medium.com/analytics-vidhya/content-based-recommender-systems-in-python-2b330e01eb80

In [33]:
desc = books.copy()
desc = desc[["Id", "Name", "Authors", "Description"]]

In [34]:
desc = desc.sample(frac=.2,
                   random_state=42)

In [35]:
desc.reset_index(inplace=True)
desc.drop("index", axis=1, inplace=True)

In [36]:
tracker = EmissionsTracker()

tracker.start()
tfidf = TfidfVectorizer(stop_words='english')
desc['Description'] = desc['Description'].fillna("")
descr_matrix = tfidf.fit_transform(desc['Description'])
similarity_matrix = linear_kernel(descr_matrix, descr_matrix)
emissions = tracker.stop()

print("GHG Emissions: " + str(emissions) + ' kg/CO2')

CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
CODECARBON : Failed to match CPU TDP constant. Falling back on a global constant.
/usr/local/lib/python3.7/dist-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/usr/local/lib/python3.7/dist-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)
/usr/local/lib/python3.7/dist-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as t

GHG Emissions: 0.0002398766175715092 kg/CO2


In [37]:
mapping = pd.Series(desc.index,
          index = desc['Name'])
print(mapping)

Name
On the Other Side of Mount Ararat: A Story of a Vanished City        0
The Tall Uncut: Stories                                              1
Heart Essence of the Vast Expanse: A Story of Transmission           2
Don't Look a Ghost Horse in the Mouth                                3
Turpentine                                                           4
                                                                 ...  
Infernal Revenue (The Destroyer, #96)                            23911
Die with Me                                                      23912
Kingyo: The Artistry of the Japanese Goldfish                    23913
The Shadow Roads (The Swans' War, book 3)                        23914
Roommates                                                        23915
Length: 23916, dtype: int64


In [38]:
def recommend_books(book_input):
    print("Book Read: ", book_input)
    book_index = mapping[book_input]
    #get similarity values with other books
    #similarity_score is the list of index and similarity matrix
    similarity_score = list(enumerate(similarity_matrix[book_index]))
    #sort in descending order the similarity score of movie inputted with all the other books
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 5 most similar books. Ignore the first book.
    similarity_score = similarity_score[1:5]
    
    #return book names using the mapping series
    book_indices = [i[0] for i in similarity_score]
    return (desc['Name'].iloc[book_indices])

In [39]:
recommend_books('The Shining')

Book Read:  The Shining


7724     Room with a Clue (Pennyfoot Hotel #1)
20610                      Buzby to the Rescue
2395      Maid to Murder (Pennyfoot Hotel #12)
12329                                   Double
Name: Name, dtype: object

In [40]:
del tracker, emissions
gc.collect()

75

### b) Using Transformers

https://towardsdatascience.com/hands-on-content-based-recommender-system-using-python-1d643bf314e4

In [41]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [42]:
data = pd.read_parquet(path+"books_with_complete_description.parquet.gzip")
data = data[["Name", "Authors", "Description"]].sample(frac=.3, random_state=42)
data.reset_index(inplace=True)
data.drop("index", axis=1, inplace=True)
data.head()

,Name,Authors,Description
0,On the Other Side of Mount Ararat: A Story of ...,Mariam Manoukian,On the Other Side of Mount Ararat is the story...
1,The Tall Uncut: Stories,Pete Fromm,"In this honest, contemporary collection of sho..."
2,Heart Essence of the Vast Expanse: A Story of ...,Anne Carolyn Klein,"Beautiful, evocative, and eminently useful, th..."
3,Don't Look a Ghost Horse in the Mouth,George E. Stanley,The horse Emily brags about to her classmates ...
4,Turpentine,Spring Warren,A comic glance at the old American West and a ...


In [43]:
X = np.array(data["Description"])

In [44]:
text_data = X

tracker = EmissionsTracker()
tracker.start()

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(text_data, show_progress_bar=True)
X = np.array(embeddings)
cos_sim_data = pd.DataFrame(cosine_similarity(X))

emissions = tracker.stop()
print("GHG Emissions: " + str(emissions) + " kg/CO2")

CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.
CODECARBON : Failed to match CPU TDP constant. Falling back on a global constant.
/usr/local/lib/python3.7/dist-packages/apscheduler/util.py:95: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  if obj.zone == 'local':
/usr/local/lib/python3.7/dist-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


Batches:   0%|          | 0/1122 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/apscheduler/triggers/interval.py:66: PytzUsageWarning: The normalize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  return self.timezone.normalize(next_fire_time)


GHG Emissions: 0.003380648796305687 kg/CO2


In [48]:
def give_recommendations(index, print_recommendation = False,print_recommendation_plots= False):
  index_recomm =cos_sim_data.loc[index].sort_values(ascending=False).index.tolist()[1:6]
  books_recomm =  data['Name'].loc[index_recomm].values
  result = {'Books':books_recomm,'Index':index_recomm}
  if print_recommendation==True:
    print('Book Read: %s \n'%(data['Name'].loc[index]))
    k=1
    for book in books_recomm:
      print('Recommended book #%i : %s \n'%(k, book))
      k = k+1
  if print_recommendation_plots==True:
    print('Book Read - Description:\n %s \n'%(data['Description'].loc[index]))
    k=1
    for q in range(len(books_recomm)):
      plot_q = data['Description'].loc[index_recomm[q]]
      print('Recommended Book #%i - Description:\n %s \n'%(k,plot_q))
      k=k+1
  return result

In [49]:
give_recommendations(10580, True, True)

Book Read: It 

Recommended book #1 : Take these Broken Wings 

Recommended book #2 : The Shining 

Recommended book #3 : The Wind Off the Sea (The Bexham Trilogy #2) 

Recommended book #4 : The Invisible Wall 

Recommended book #5 : The Knife of Never Letting Go (Chaos Walking, #1) 

Book Read - Description:
 To the children, the town was their whole world. To the adults, knowing better, Derry, Maine was just their home town: familiar, well-ordered for the most part. A good place to live.<br /><br />It was the children who saw - and felt - what made Derry so horribly different. In the storm drains, in the sewers, IT lurked, taking on the shape of every nightmare, each one's deepest dread. Sometimes IT reached up, seizing, tearing, killing...<br /><br />The adults, knowing better, knew nothing.<br /><br />Time passed and the children grew up, moved away. The horror of IT was deep-buried, wrapped in forgetfulness. Until they were called back, once more to confront IT as it stirred and c

{'Books': array(['Take these Broken Wings', 'The Shining',
        'The Wind Off the Sea (The Bexham Trilogy #2)',
        'The Invisible Wall',
        'The Knife of Never Letting Go (Chaos Walking, #1)'], dtype=object),
 'Index': [22887, 1952, 948, 28447, 31252]}

In [50]:
recomm_list = []

for i in range(len(X)):
  recomm_i = give_recommendations(i)
  recomm_list.append(recomm_i['Books'])
recomm_data = pd.DataFrame(recomm_list,columns=['First Recommendation',
                                                'Second Recommendation',
                                                'Third Recommendation',
                                                'Fourth Recommendation',
                                                'Fifth Recommendation'])
recomm_data['Books Read'] = data['Name']
recomm_data = recomm_data[['Books Read',
                           'First Recommendation',
                           'Second Recommendation',
                           'Third Recommendation',
                           'Fourth Recommendation',
                           'Fifth Recommendation']]

In [51]:
recomm_data.sample(frac=1).head()

,Books Read,First Recommendation,Second Recommendation,Third Recommendation,Fourth Recommendation,Fifth Recommendation
11212,The First Dog,The Berenstain Bears Blaze a Trail,The Magic Gourd,"Yum, Yum, Yummy! (The Giggle Club)",The Great Tug of War,"What Time Is It, Mr. Crocodile?"
2548,Revelations Cycle II: The Marches (In Nomine: ...,Bone Volume 6: Old Man's Cave,Gregor and the Code of Claw (Underland Chronic...,Midnight (Midnight),"Winterbirth (The Godless World, #1)","Destiny's Truth (Deathlands, #60)"
6010,"Methods in Cell Biology, Volume 63: Cytometry,...","Methods in Cell Biology, Volume 63: Cytometry,...",Knowledge of Life,Avery's Diseases Of The Newborn,Borderline Personality Disorder: Clinical and ...,Case Studies in Immunology: A Clinical Companion
25380,Oxford Atlas of the World,Emerging Powers (The Illustrated History of th...,A History of the World in the Twentieth Century,An Incomplete Education,An Incomplete Education,Roots: The Saga of an American Family
10621,Just Us Girls: Secrets to Feeling Good About Y...,GirlForce: Vibe: A Girl's Guidebook to Confide...,My Own Thoughts and Feelings (for Girls): A Yo...,Guys Like Girls Who . . .,Changing Your World One Diaper at a Time: A Re...,It's a Girl: Women Writers on Raising Daughters


In [52]:
del users,\
    books,\
    data
gc.collect()

557

**References:**
- https://pixelfridge.digital/the-paradox-of-choice-why-more-isnt-always-better/